<a href="https://colab.research.google.com/github/Ng-Cho-Yin/YOLO_v5/blob/main/yolov5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from zipfile import ZipFile

In [2]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [3]:
with ZipFile('/content/drive/MyDrive/global-wheat-detection.zip','r') as zipObj:
  zipObj.extractall('wheat_data/input')

In [4]:
with ZipFile('/content/drive/MyDrive/yolov5-master.zip','r') as zipObj:
  zipObj.extractall('yolo')

In [5]:
import ast
import numpy as np
import os
import pandas as pd
from PIL import Image
from sklearn import model_selection
import shutil
from tqdm import tqdm




In [ ]:
#shutil.rmtree('/content/wheat_data/input', ignore_errors=True)

In [6]:
DATA_PATH='/content/wheat_data/input'
OUTPUT_PATH='/content/yolo/yolov5-master/wheat_data'

In [ ]:
df = pd.read_csv(os.path.join(DATA_PATH,'train.csv'))
df.bbox = df.bbox.apply(ast.literal_eval)
df = df.groupby("image_id")['bbox'].apply(list).reset_index(name='bboxes')

In [ ]:
df

,image_id,bboxes
0,00333207f,"[[0, 654, 37, 111], [0, 817, 135, 98], [0, 192..."
1,005b0d8bb,"[[765.0, 879.0, 116.0, 79.0], [84.0, 539.0, 15..."
2,006a994f7,"[[437.0, 988.0, 98.0, 36.0], [309.0, 527.0, 11..."
3,00764ad5d,"[[89.0, 256.0, 113.0, 107.0], [216.0, 282.0, 1..."
4,00b5fefed,"[[709.0, 97.0, 204.0, 105.0], [775.0, 250.0, 1..."
...,...,...
3368,ffb445410,"[[0.0, 534.0, 54.0, 118.0], [0.0, 480.0, 38.0,..."
3369,ffbf75e5b,"[[0, 697, 21, 58], [104, 750, 77, 75], [65, 84..."
3370,ffbfe7cc0,"[[256.0, 0.0, 64.0, 99.0], [390.0, 0.0, 48.0, ..."
3371,ffc870198,"[[447.0, 976.0, 78.0, 48.0], [18.0, 141.0, 218..."


In [ ]:
df_train,df_valid = model_selection.train_test_split(
                    df,
                    test_size=0.1,
                    random_state=42,
                    shuffle=True)
df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)

In [ ]:
df_train

,image_id,bboxes
0,6d34eb3b2,"[[391, 760, 87, 80], [208, 400, 149, 79], [215..."
1,c7c00c1a3,"[[91, 10, 91, 66], [753, 0, 58, 38], [570, 0, ..."
2,09b4141a6,"[[245.0, 240.0, 113.0, 110.0], [296.0, 709.0, ..."
3,e837f4e67,"[[491.0, 473.0, 159.0, 87.0], [558.0, 225.0, 8..."
4,f1d20d58a,"[[459.0, 917.0, 153.0, 103.0], [684.0, 5.0, 60..."
...,...,...
3030,536ef8d03,"[[483.0, 4.0, 144.0, 92.0], [50.0, 246.0, 150...."
3031,572a469e9,"[[458.0, 619.0, 109.0, 90.0], [825.0, 604.0, 8..."
3032,63c4e8cda,"[[84, 83, 190, 123], [260, 56, 181, 145], [7, ..."
3033,444977945,"[[542.0, 0.0, 87.0, 46.0], [529.0, 175.0, 90.0..."


In [ ]:
img = Image.open('/content/wheat_data/input/train/00333207f.jpg')

In [ ]:
img.size

(1024, 1024)

In [ ]:
def process_data(data,data_type='train'):
    for _,row in tqdm(data.iterrows(),total=len(data)):
        image_name = row['image_id']
        bounding_boxes = row['bboxes']
        yolo_data = []
        for bbox in bounding_boxes:
            x = bbox[0]
            y = bbox[1]
            w = bbox[2]/1024
            h = bbox[3]/1024
            x_center = (x+w/2)/1024
            y_center = (y+h/2)/1024
            yolo_data.append([0,x_center,y_center,w,h])#['class',x_center,y_center,w,h]
        yolo_data = np.array(yolo_data)
        np.savetxt(
            os.path.join(OUTPUT_PATH,f"labels/{data_type}/{image_name}.txt"),
            yolo_data,
            fmt=['%d','%f','%f','%f','%f']
        )
        shutil.copyfile(
            os.path.join(DATA_PATH,f"train/{image_name}.jpg"),
            os.path.join(OUTPUT_PATH,f"images/{data_type}/{image_name}.jpg"),
        )

In [ ]:
process_data(df_train,data_type='train')
process_data(df_valid,data_type='validation')

100%|██████████| 338/338 [00:00<00:00, 879.41it/s]


In [ ]:
!pip install -r /content/yolo/yolov5-master/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 7.5 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
! python /content/yolo/yolov5-master/train.py --img 1024 --batch 8 --epochs 20 --data /content/yolo/yolov5-master/wheat.yaml --cfg /content/yolo/yolov5-master/models/yolov5s.yaml --name wm_20e

python3: can't open file '/content/yolo/yolov5-master/train.py': [Errno 2] No such file or directory


In [ ]:
! python /content/yolo/yolov5-master/detect.py --source /content/wheat_data/input/test --weights /content/yolo/yolov5-master/runs/train/wm2/weights/best.pt

detect: weights=['/content/yolo/yolov5-master/runs/train/wm2/weights/best.pt'], source=/content/wheat_data/input/test, data=yolo/yolov5-master/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolo/yolov5-master/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 2022-7-3 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/10 /content/wheat_data/input/test/2fd875eaa.jpg: 640x640 41 wheats, Done. (0.014s)
image 2/10 /content/wheat_data/input/test/348a992bb.jpg: 640x640 55 wheats, Done. (0.013s)
image 3/10 /content/wheat_data/input/test/51b3e36ab.jpg: 640x640 28 wheats, Done. (0.013s)
image 4/10 /conte